In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft.service.worker.worker_image import SyftWorkerImage, SyftWorkerImageTag
from syft.custom_worker.config import DockerWorkerConfig
from syft.service.worker.worker_image import build_using_docker
from syft.service.worker.utils import run_container_using_docker


#third party
import docker

In [ ]:
domain  = sy.orchestra.launch(name="test-domain-1", reset=True, dev_mode=True, port="auto")

In [ ]:
domain_client = domain.login(email="info@openmined.org", password="changethis")

In [ ]:
nginx_dockerfile_str = """
# Use the official Nginx image as the base
FROM nginx:latest

# Expose port 80 for the Nginx server
EXPOSE 80

# Start Nginx when the container has provisioned
CMD ["nginx", "-g", "daemon off;"]
"""

In [ ]:
docker_config = DockerWorkerConfig(dockerfile=nginx_dockerfile_str)

In [ ]:
assert docker_config.dockerfile == nginx_dockerfile_str

In [ ]:
res = domain_client.api.services.worker_image.submit_dockerfile(docker_config=docker_config)

In [ ]:
res

In [ ]:
assert isinstance( res, sy.SyftSuccess)

In [ ]:
dockerfile_list = domain_client.api.services.worker_image.get_all()

In [ ]:
assert len(dockerfile_list) == 1

In [ ]:
workerimage = dockerfile_list[0]
workerimage

In [ ]:
assert isinstance(workerimage, SyftWorkerImage)
assert workerimage.config.dockerfile == nginx_dockerfile_str

In [ ]:
docker_tag = "openmined/test-nginx:0.7.8"
docker_build_res = domain_client.api.services.worker_image.build(uid=workerimage.id, tag=docker_tag)

In [ ]:
docker_build_res

In [ ]:
assert isinstance(docker_build_res, sy.SyftSuccess)

In [ ]:
import subprocess

def check_image_exists(tag) -> bool:
    result = subprocess.run(['docker', 'images', '-q', tag], stdout=subprocess.PIPE)
    return result.stdout.strip() != b''
assert check_image_exists(docker_tag)

In [ ]:
image_list = domain_client.api.services.worker_image.get_all()
image_list

In [ ]:
assert len(image_list) == 1

In [ ]:
import docker

def get_image_hash(tag) -> str:
    client = docker.from_env()
    try:
        image = client.images.get(tag)
        return image.id
    except docker.errors.ImageNotFound:
        return None

In [ ]:
assert image_list[0].image_hash == get_image_hash(docker_tag)

In [ ]:
def get_container_id(container_name: str) -> str:
    client = docker.from_env()
    try:
        container = client.containers.get(container_name)
        return container.id
    except docker.errors.NotFound:
        return None

In [ ]:
worker_pool_name = "my_first_worker_pool"
worker_pool_res = domain_client.api.services.worker_pool.create(name=worker_pool_name, image_uid = image_list[0].id, number=3)

In [ ]:
worker_pool_res

In [ ]:
assert len(worker_pool_res) == 3

In [ ]:
for status in worker_pool_res:
    assert status.error == None
    assert status.worker.image_hash == get_image_hash(docker_tag)

In [ ]:
worker_pool_list = domain_client.api.services.worker_pool.get_all()

In [ ]:
worker_pool_list

In [ ]:
assert len(worker_pool_list)==1
worker_pool = worker_pool_list[0]
assert worker_pool.name==worker_pool_name
assert len(worker_pool.workers)==3

In [ ]:
# Delete the second worker
second_worker = worker_pool.workers[1]

In [ ]:
worker_delete_res = domain_client.api.services.worker_pool.delete_worker(worker_pool_id=worker_pool.id,
                                                                         worker_id=second_worker.id)

In [ ]:
worker_delete_res

In [ ]:
assert isinstance(worker_delete_res,sy.SyftSuccess)

In [ ]:
# Refetch the worker pool
# Ensure that the deleted worker's id is not present
worker_pool = domain_client.api.services.worker_pool.get_all()[0]
assert len(worker_pool.workers)==2
for worker in worker_pool.workers:
    assert second_worker.id != worker.id

In [ ]:
worker_pool

In [ ]:
delete_res = domain_client.api.services.worker_image.delete(workerimage.id)

In [ ]:
# Since the containers are running, we would not able to delete the image
assert isinstance(delete_res, sy.SyftError)
delete_res

In [ ]:
# delete the remaining workers
for worker in worker_pool.workers:
    res =domain_client.api.services.worker_pool.delete_worker(worker_pool_id=worker_pool.id,
                                                         worker_id= worker.id)
    assert isinstance(res, sy.SyftSuccess)

In [ ]:
delete_res = domain_client.api.services.worker_image.delete(workerimage.id)
delete_res

In [ ]:
# Since the containers are delete, we should be able to delete the image
assert isinstance(delete_res, sy.SyftSuccess)
delete_res